In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

In [ ]:
# to see how many responses i get , must be 372 as it said total 372 entries in first request
url="https://api.github.com/search/users"
TOKEN = 'had to remove '
HEADERS = {'Authorization': f'token {TOKEN}'}
for page in range(1,5):
    query = 'location:Sydney followers:>100'
    params = {'q': query, 'per_page': 100, 'page': page}
    res = requests.get(url, headers=HEADERS, params=params)
    print(len(res.json()['items']))
    print('--------------------------------------------------------------------------------')
    time.sleep(1)

In [ ]:
# check the format of each entry
url="https://api.github.com/search/users"
TOKEN = 'had to remove'
HEADERS = {'Authorization': f'token {TOKEN}'}
query = 'location:Sydney followers:>100'
params = {'q': query, 'per_page': 100, 'page': 1}
#check if url in one entry gives all info
res = requests.get(url, headers=HEADERS, params=params)
data=res.json()
url=data['items'][1]['url']
data_of_usr1=requests.get(url)
data_of_usr1.json()

In [ ]:
url = "https://api.github.com/search/users"
TOKEN = 'removed'
HEADERS = {'Authorization': f'token {TOKEN}'}
query = 'location:Sydney followers:>100'
users_data = []
repositories_data = []

In [17]:
for page in range(1, 5):  # only 372 entries, so 1,2,3 pages have 100 and 4th one have 72
    params = {'q': query, 'per_page': 100, 'page': page}
    res = requests.get(url, headers=HEADERS, params=params)

    if res.status_code == 200:
        # Extract each user in the search results
        users = res.json()['items']
        for user in users:
            user_url = user['url']  # URL for the full profile data
            user_res = requests.get(user_url, headers=HEADERS)

            if user_res.status_code == 200:
                user_details = user_res.json()

                # Parse and clean up user details
                user_data = {
                    "login": user_details['login'],
                    "name": user_details.get('name', '') or '',
                    "company": (user_details.get('company', '') or '').strip('@').upper(),
                    "location": user_details.get('location', '') or '',
                    "email": user_details.get('email', '') or '',
                    "hireable": str(user_details.get('hireable', '')).lower() if user_details.get('hireable') is not None else '',
                    "bio": user_details.get('bio', '') or '',
                    "public_repos": user_details.get('public_repos', 0),
                    "followers": user_details['followers'],
                    "following": user_details['following'],
                    "created_at": user_details['created_at']
                }
                users_data.append(user_data)

                # Get up to 500 repositories for each user
                repos_url = f"https://api.github.com/users/{user_details['login']}/repos"
                for repo_page in range(1, 6):  # 100 repos per page, so 5*100 = 500 which was needed
                    repo_params = {'sort': 'pushed', 'per_page': 100, 'page': repo_page}
                    repo_res = requests.get(repos_url, headers=HEADERS, params=repo_params)

                    if repo_res.status_code == 200:
                        repos = repo_res.json()
                        if not repos:
                            break  # Exit if no more repos
                        for repo in repos:
                            repo_data = {
                                "login": user_details['login'],
                                "full_name": repo['full_name'],
                                "created_at": repo['created_at'],
                                "stargazers_count": repo['stargazers_count'],
                                "watchers_count": repo['watchers_count'],
                                "language": repo.get('language', '') or '',
                                "has_projects": str(repo['has_projects']).lower(),
                                "has_wiki": str(repo['has_wiki']).lower(),
                                "license_name": repo['license']['key'] if repo['license'] else ''
                            }
                            repositories_data.append(repo_data)
                    else:
                        print(f"Failed to fetch repositories for user {user_details['login']} on page {repo_page}")

            else:
                print(f"Failed to fetch details for user {user['login']}")

    else:
        print(f"Failed to fetch users on page {page}")

Data collection complete. Check 'users.csv' and 'repositories.csv' for results.


In [ ]:
users_df = pd.DataFrame(users_data)
repositories_df = pd.DataFrame(repositories_data)
print("Data collection complete.")

In [18]:
users_df

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
0,nicknochnack,Nicholas Renotte,,"Sydney, Australia",,,Data Science guy @IBM,193,11678,5,2013-11-15T15:06:35Z
1,brendangregg,Brendan Gregg,INTEL,"Sydney, Australia",,,Cloud computing performance engineer.,42,8022,0,2011-10-04T08:48:56Z
2,cornflourblue,Jason Watmore,POINT BLANK DEVELOPMENT,Sydney Australia,,,"Web Developer and Blogger, currently focusing ...",187,6700,0,2011-05-13T04:38:16Z
3,0vm,Ak,,"Sydney, Australia",,true,erm hai,10,6568,3,2021-03-02T06:55:10Z
4,davecheney,Dave Cheney,GITHUB,"Sydney, Australia",,,I service society by rocking.,163,6557,42,2008-04-14T04:00:20Z
...,...,...,...,...,...,...,...,...,...,...,...
367,ftsf,impbox,IMPBOX GAMES,"Sydney, Australia",,true,"Indie Game Developer, Cat Photographer, former...",26,102,4,2009-10-19T02:39:13Z
368,dhowden,David Howden,ALGOLIA (PREV @SAJARI),"Sydney, Australia",dhowden@gmail.com,,,41,101,32,2008-03-11T15:58:04Z
369,peterboyer,Peter Boyer,,"Sydney, Australia",,true,,20,101,24,2014-08-08T06:02:54Z
370,tomitrescak,Tomas Trescak,WESTERN SYDNEY UNIVERSITY,"Sydney, Australia",,,,184,101,2,2012-10-30T10:36:25Z


In [19]:
repositories_df

,login,full_name,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license_name
0,nicknochnack,nicknochnack/TechXAIStockAgent,2024-10-23T23:48:51Z,4,4,Python,true,true,
1,nicknochnack,nicknochnack/DataFrameAgentWatsonx,2024-10-04T03:37:19Z,6,6,Python,true,true,
2,nicknochnack,nicknochnack/jlite,2024-09-27T23:36:49Z,1,1,Jupyter Notebook,true,true,
3,nicknochnack,nicknochnack/beeagent,2024-09-19T08:05:08Z,8,8,TypeScript,true,true,
4,nicknochnack,nicknochnack/WatsonxFinCom,2024-09-02T04:26:15Z,3,3,Python,true,true,
...,...,...,...,...,...,...,...,...,...
32436,chadluo,chadluo/LEMONADE-LICENSE,2013-08-14T09:42:06Z,0,0,,true,true,other
32437,chadluo,chadluo/wedding-party-speech,2013-07-24T20:01:19Z,0,0,,true,true,
32438,chadluo,chadluo/music-theory,2013-07-29T14:33:38Z,17,17,,true,true,
32439,chadluo,chadluo/COMP2911-Project,2014-05-06T06:22:38Z,0,0,Java,true,true,


In [20]:
users_df.to_csv('users.csv', index=False)
repositories_df.to_csv('repositories.csv', index=False)